In [211]:
from selenium import webdriver
from selenium.webdriver.common.by import By  # Importação correta do By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager  # Importação do ChromeDriverManager
import time
import os
import pandas as pd
import numpy as np

servico = Service(ChromeDriverManager().install())

# Configuração do diretório de download
download_directory = os.path.join(os.getcwd(), "dados")
if not os.path.exists(download_directory):  # Cria a pasta se não existir
    os.makedirs(download_directory)

# Opções do Chrome
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_directory,  # Define a pasta de download
    "download.prompt_for_download": False,  # Não solicita confirmação para downloads
    "directory_upgrade": True,
})

In [212]:
# Dicionário com as marcas e seus valores
dados = {
    "Marca": [
        "Itaú", "Banco do Brasil", "Bradesco", "Petrobras", "Vale", 
        "Caixa", "Localiza", "Brahma", "Natura", "Skol", 
        "Sadia", "Vivo", "Rede D’or São Luiz", "Nubank", "Marfrig", 
        "Latam Airlines", "Assai Atacadista", "Raízen", "Magazine Luiza", "Antarctica", 
        "Suzano", "Ipiranga", "Embraer", "Braskem", "Atacadão", 
        "Unidas", "Sabesp", "Klabin", "Movida", "PagSeguro Digital", 
        "Renner", "Camil", "Votorantim", "Globo", "Vibra", 
        "XP Inc.", "BNDES", "Gerdau", "Seguradora Porto", "Itambé", 
        "SmartFit", "Minerva", "Stone", "Perdigão", "Neoenergia", 
        "Hapvida", "B3", "Sul América", "Ypióca", "Banco Pan"
    ],
    "Valor": [
        8330000000, 5450000000, 5010000000, 3400000000, 3000000000, 
        2860000000, 2300000000, 1960000000, 1950000000, 1900000000, 
        1800000000, 1700000000, 1370000000, 1350000000, 1350000000, 
        1220000000, 1200000000, 1090000000, 994000000, 992000000, 
        920000000, 918000000, 864000000, 827000000, 820000000, 
        766000000, 751000000, 693000000, 682000000, 630000000, 
        615000000, 574000000, 569000000, 558000000, 553000000, 
        544000000, 543000000, 536000000, 500000000, 444000000, 
        432000000, 426000000, 416000000, 390000000, 388000000, 
        377000000, 358000000, 357000000, 354000000, 342000000
    ]
}

## Fazer Web Scraping das Marcas

In [217]:
driver = webdriver.Chrome(service=servico, options=options)

# Acessa o Google Trends
driver.get("https://trends.google.com/trends/explore?geo=BR&hl=pt-BR")

# Espera alguns segundos para garantir o carregamento da página
time.sleep(2)

# Recarrega a página
driver.refresh()

# Espera mais alguns segundos após a recarga
time.sleep(2)

# Inicializa ActionChains
actions = ActionChains(driver)

# Dicionário de XPaths dos botões
buttons = {
    "Tempo": '/html/body/div[3]/div[2]/div/md-content/div/div/div[1]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]/i',
    "Regiao": '/html/body/div[3]/div[2]/div/md-content/div/div/div[2]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]/i'
}

def click_button(button_xpath):
    """
    Clica em um botão baseado no XPath fornecido, rolando a página até o botão.
    
    :param button_xpath: O XPath do botão a ser clicado.
    """
    button = driver.find_element(By.XPATH, button_xpath)
    driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });", button)
    time.sleep(1)
    actions.move_to_element(button).click().perform()

# Percorre cada marca no DataFrame
for marca in dados["Marca"]:
    print(f"Pesquisando por: {marca}")

    # Volta para o topo da página
    driver.execute_script("window.scrollTo(0, 0);")

    # Pesquisa a marca
    search_box_xpath = '//*[@id="input-29"]'  # XPath da caixa de busca
    search_box = driver.find_element(By.XPATH, search_box_xpath)

    # Limpa a caixa de busca usando BACKSPACE
    search_box_value = search_box.get_attribute('value')  # Obtém o valor atual da caixa
    for _ in range(len(search_box_value)):
        search_box.send_keys(Keys.BACKSPACE)  # Remove um caractere por vez
    
    # Digita a nova marca e submete a busca
    search_box.send_keys(marca)
    search_box.send_keys(Keys.RETURN)

    # Espera o carregamento dos dados
    time.sleep(3)
    
    # Itera sobre os botões e clica em cada um
    for name, xpath in buttons.items():
        click_button(xpath)  # Clica no botão

        # Aguarda o download
        print(f"Aguardando o download de {name} para {marca}...")
        time.sleep(3)  # Aumentando o tempo de espera para garantir o download

    # Espera até que ambos os arquivos tenham sido baixados
    multi_timeline_path = os.path.join(download_directory, 'multiTimeline.csv')
    geo_map_path = os.path.join(download_directory, 'geoMap.csv')

    while not (os.path.exists(multi_timeline_path) and os.path.exists(geo_map_path)):
        time.sleep(1)  # Aguarda um segundo até que ambos os arquivos sejam encontrados

    # Renomeia os arquivos
    new_multi_timeline_path = os.path.join(download_directory, f"{marca}_Tempo.csv")
    new_geo_map_path = os.path.join(download_directory, f"{marca}_Regiao.csv")

    if not os.path.exists(new_multi_timeline_path):
        os.rename(multi_timeline_path, new_multi_timeline_path)  # Renomeia o arquivo multiTimeline

    if not os.path.exists(new_geo_map_path):
        os.rename(geo_map_path, new_geo_map_path)  # Renomeia o arquivo geoMap

    print(f"Download e renomeação concluídos para: {marca} - Tempo e Regiao")

    # Aguarda um pouco antes de começar a nova pesquisa
    time.sleep(2)

# Fecha o navegador ao final
driver.quit()

Pesquisando por: Itaú
Aguardando o download de Tempo para Itaú...
Aguardando o download de Regiao para Itaú...
Download e renomeação concluídos para: Itaú - Tempo e Regiao
Pesquisando por: Banco do Brasil
Aguardando o download de Tempo para Banco do Brasil...
Aguardando o download de Regiao para Banco do Brasil...
Download e renomeação concluídos para: Banco do Brasil - Tempo e Regiao
Pesquisando por: Bradesco
Aguardando o download de Tempo para Bradesco...
Aguardando o download de Regiao para Bradesco...
Download e renomeação concluídos para: Bradesco - Tempo e Regiao
Pesquisando por: Petrobras
Aguardando o download de Tempo para Petrobras...
Aguardando o download de Regiao para Petrobras...
Download e renomeação concluídos para: Petrobras - Tempo e Regiao
Pesquisando por: Vale
Aguardando o download de Tempo para Vale...
Aguardando o download de Regiao para Vale...
Download e renomeação concluídos para: Vale - Tempo e Regiao
Pesquisando por: Caixa
Aguardando o download de Tempo para C

## Juntar os arquivos Semana

In [469]:
# Lista para armazenar os DataFrames
dfs_Tempo = pd.DataFrame()
dfs_Tempo["Marca"] = dados["Marca"]

df_Tempo = pd.read_csv("dados/Skol_Tempo.csv", skiprows=1)

# Obtém as datas da coluna 'Semana'
datas = df_Tempo["Semana"].tolist()

# Cria novas colunas em dfs para cada data
for data in datas:
    coluna_nome = f"{data}"
    dfs_Tempo[coluna_nome] = np.nan  # Preenche com NaN

# Exemplo de como o diretório de arquivos CSV pode ser configurado
download_directory = "dados"

# Itera sobre cada marca no DataFrame
for marca in dfs_Tempo["Marca"]:
    print(f"Processando a marca: {marca}")

    # Cria o nome do arquivo CSV correspondente
    arquivo_csv = os.path.join(download_directory, f"{marca}_Tempo.csv")
    
    # Verifica se o arquivo existe
    if os.path.exists(arquivo_csv):
        # Lê o arquivo CSV
        df_temp = pd.read_csv(arquivo_csv, skiprows=1)
        
        # Itera sobre cada data na coluna 'Semana' de dfs
        for data in dfs_Tempo.columns[1:]:  # Ignora a primeira coluna 'Marca'
            # Nome da coluna correspondente no df_temp
            coluna_nome_temp = f"{marca}: (Brasil)"  # O formato do nome da coluna

            # Verifica se a coluna existe no df_temp
            if coluna_nome_temp in df_temp.columns:
                # Obtém o valor correspondente da coluna de df_temp
                valor = df_temp.loc[df_temp["Semana"] == data, coluna_nome_temp].values
                
                if valor.size > 0:  # Verifica se encontrou algum valor
                    # Preenche o valor na coluna correspondente em dfs
                    dfs_Tempo.loc[dfs_Tempo["Marca"] == marca, data] = valor[0]  # Atribui o valor

Processando a marca: Itaú
Processando a marca: Banco do Brasil
Processando a marca: Bradesco
Processando a marca: Petrobras
Processando a marca: Vale
Processando a marca: Caixa
Processando a marca: Localiza
Processando a marca: Brahma
Processando a marca: Natura
Processando a marca: Skol
Processando a marca: Sadia
Processando a marca: Vivo
Processando a marca: Rede D’or São Luiz
Processando a marca: Nubank
Processando a marca: Marfrig
Processando a marca: Latam Airlines
Processando a marca: Assai Atacadista
Processando a marca: Raízen
Processando a marca: Magazine Luiza
Processando a marca: Antarctica
Processando a marca: Suzano
Processando a marca: Ipiranga
Processando a marca: Embraer
Processando a marca: Braskem
Processando a marca: Atacadão
Processando a marca: Unidas
Processando a marca: Sabesp
Processando a marca: Klabin
Processando a marca: Movida
Processando a marca: PagSeguro Digital
Processando a marca: Renner
Processando a marca: Camil
Processando a marca: Votorantim
Process

In [470]:
# Preenche NaNs com 0.0
dfs_Tempo.fillna(0.0, inplace=True)

# Transforma a coluna 'Marca' em índice
dfs_Tempo.set_index("Marca", inplace=True)

# Renomeia o índice, removendo a palavra "Marca"
dfs_Tempo.index.name = None  # Remove o nome do índice

dfs_Tempo= dfs_Tempo.astype(int)

In [473]:
dfs_Tempo

,2023-10-15,2023-10-22,2023-10-29,2023-11-05,2023-11-12,2023-11-19,2023-11-26,2023-12-03,2023-12-10,2023-12-17,...,2024-08-11,2024-08-18,2024-08-25,2024-09-01,2024-09-08,2024-09-15,2024-09-22,2024-09-29,2024-10-06,2024-10-13
Itaú,81,66,81,87,71,72,83,97,94,85,...,75,72,76,92,73,73,71,89,82,85
Banco do Brasil,82,74,73,89,71,76,79,86,88,87,...,79,79,76,86,83,79,75,81,79,88
Bradesco,78,80,73,81,69,74,97,79,88,77,...,73,74,73,84,77,70,69,78,72,74
Petrobras,26,29,22,26,21,22,20,18,21,20,...,24,24,23,22,21,20,22,24,22,24
Vale,88,90,87,88,89,87,85,88,95,91,...,86,85,86,88,85,85,87,85,88,94
Caixa,73,66,66,77,69,74,69,72,80,74,...,69,67,64,66,73,65,65,66,65,67
Localiza,80,81,79,83,92,83,83,89,94,97,...,76,71,75,78,80,71,77,75,74,80
Brahma,40,41,42,42,50,43,48,58,52,61,...,41,39,37,38,37,34,35,35,33,34
Natura,70,68,66,75,77,89,83,97,100,100,...,69,62,63,66,66,69,66,65,61,68
Skol,49,50,48,53,59,51,48,63,70,76,...,45,45,48,48,48,47,45,47,45,48


## Juntar os arquivos Regiao

In [461]:
# Lista para armazenar os DataFrames
dfs_Regiao = pd.DataFrame()
dfs_Regiao["Marca"] = dados["Marca"]

df_Regiao = pd.read_csv("dados/Skol_Regiao.csv", skiprows=1)

# Obtém as regiões da coluna 'Região'
regioes = df_Regiao["Região"].tolist()

# Cria novas colunas em dfs para cada região
for regiao in regioes:
    coluna_nome = f"{regiao}"
    dfs_Regiao[coluna_nome] = np.nan  # Preenche com NaN

# Exemplo de como o diretório de arquivos CSV pode ser configurado
download_directory = "dados"

# Itera sobre cada marca no DataFrame
for marca in dfs_Regiao["Marca"]:
    print(f"Processando a marca: {marca}")

    # Cria o nome do arquivo CSV correspondente
    arquivo_csv = os.path.join(download_directory, f"{marca}_Regiao.csv")
    
    # Verifica se o arquivo existe
    if os.path.exists(arquivo_csv):
        # Lê o arquivo CSV
        df_regiao = pd.read_csv(arquivo_csv, skiprows=1)
        
        # Itera sobre cada região na coluna 'Região' de dfs
        for regiao in dfs_Regiao.columns[1:]:  # Ignora a primeira coluna 'Marca'
            # Nome da coluna correspondente no df_regiao
            coluna_nome_regiao = f"{marca}: (16/10/2023 – 16/10/2024)"  # O formato do nome da coluna
            
            # Verifica se a coluna existe no df_regiao
            if coluna_nome_regiao in df_regiao.columns:
                # Obtém o valor correspondente da coluna de df_regiao
                valor = df_regiao.loc[df_regiao["Região"] == regiao, coluna_nome_regiao].values
                
                if valor.size > 0:  # Verifica se encontrou algum valor
                    try:
                        # Tenta converter o valor para float
                        valor_convertido = float(valor[0])
                    except ValueError:
                        # Em caso de erro na conversão, define como NaN
                        valor_convertido = np.nan
                    
                    # Preenche o valor na coluna correspondente em dfs
                    dfs_Regiao.loc[dfs_Regiao["Marca"] == marca, regiao] = valor_convertido  # Atribui o valor

Processando a marca: Itaú
Processando a marca: Banco do Brasil
Processando a marca: Bradesco
Processando a marca: Petrobras
Processando a marca: Vale
Processando a marca: Caixa
Processando a marca: Localiza
Processando a marca: Brahma
Processando a marca: Natura
Processando a marca: Skol
Processando a marca: Sadia
Processando a marca: Vivo
Processando a marca: Rede D’or São Luiz
Processando a marca: Nubank
Processando a marca: Marfrig
Processando a marca: Latam Airlines
Processando a marca: Assai Atacadista
Processando a marca: Raízen
Processando a marca: Magazine Luiza
Processando a marca: Antarctica
Processando a marca: Suzano
Processando a marca: Ipiranga
Processando a marca: Embraer
Processando a marca: Braskem
Processando a marca: Atacadão
Processando a marca: Unidas
Processando a marca: Sabesp
Processando a marca: Klabin
Processando a marca: Movida
Processando a marca: PagSeguro Digital
Processando a marca: Renner
Processando a marca: Camil
Processando a marca: Votorantim
Process

In [465]:
# Substitui valores nulos por 0
dfs_Regiao.fillna(0, inplace=True)

# Transforma a coluna 'Marca' em índice
dfs_Regiao.set_index("Marca", inplace=True)

# Renomeia o índice, removendo a palavra "Marca"
dfs_Regiao.index.name = None  # Remove o nome do índice

# Converte todas as colunas, exceto a primeira, para inteiro
dfs_Regiao = dfs_Regiao.astype(int)

In [467]:
dfs_Regiao

,Piauí,Amapá,Pará,Maranhão,Rondônia,Roraima,Sergipe,Ceará,Mato Grosso,Tocantins,...,Mato Grosso do Sul,Amazonas,Pernambuco,Espírito Santo,Goiás,Rio de Janeiro,Distrito Federal,Paraná,Rio Grande do Sul,Santa Catarina
Itaú,32,42,45,31,29,41,39,45,31,32,...,33,45,52,45,72,100,60,59,37,38
Banco do Brasil,94,80,62,79,80,81,65,63,99,100,...,92,46,63,51,70,50,98,58,59,65
Bradesco,33,29,46,45,42,31,38,57,53,56,...,57,78,54,33,49,100,50,48,33,40
Petrobras,18,20,21,15,21,17,75,26,16,18,...,18,28,35,66,20,100,41,30,30,24
Vale,68,56,84,85,86,46,54,70,67,58,...,62,59,71,96,69,80,73,78,83,85
Caixa,89,71,79,78,82,72,90,76,79,82,...,84,73,89,84,100,77,97,81,82,79
Localiza,56,56,60,66,51,55,42,49,77,64,...,69,45,48,79,70,81,67,65,71,79
Brahma,33,21,27,43,56,54,35,29,68,41,...,60,34,46,45,100,82,45,61,47,47
Natura,93,77,86,83,63,73,87,89,57,74,...,57,72,97,68,56,69,60,54,60,48
Skol,100,89,89,75,74,73,71,65,63,63,...,52,49,47,47,40,39,36,36,32,30


## Apagar todos os arquivos

In [483]:
# Apagar todos os arquivos CSV no diretório
for filename in os.listdir(download_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(download_directory, filename)
        os.remove(file_path)
        print(f"Arquivo removido: {file_path}")

Arquivo removido: dados\Antarctica_Regiao.csv
Arquivo removido: dados\Antarctica_Tempo.csv
Arquivo removido: dados\Assai Atacadista_Regiao.csv
Arquivo removido: dados\Assai Atacadista_Tempo.csv
Arquivo removido: dados\Atacadão_Regiao.csv
Arquivo removido: dados\Atacadão_Tempo.csv
Arquivo removido: dados\B3_Regiao.csv
Arquivo removido: dados\B3_Tempo.csv
Arquivo removido: dados\Banco do Brasil_Regiao.csv
Arquivo removido: dados\Banco do Brasil_Tempo.csv
Arquivo removido: dados\Banco Pan_Regiao.csv
Arquivo removido: dados\Banco Pan_Tempo.csv
Arquivo removido: dados\BNDES_Regiao.csv
Arquivo removido: dados\BNDES_Tempo.csv
Arquivo removido: dados\Bradesco_Regiao.csv
Arquivo removido: dados\Bradesco_Tempo.csv
Arquivo removido: dados\Brahma_Regiao.csv
Arquivo removido: dados\Brahma_Tempo.csv
Arquivo removido: dados\Braskem_Regiao.csv
Arquivo removido: dados\Braskem_Tempo.csv
Arquivo removido: dados\Caixa_Regiao.csv
Arquivo removido: dados\Caixa_Tempo.csv
Arquivo removido: dados\Camil_Regiao.

## Salvar os DFs

In [485]:
# Corrige a chamada para salvar o DataFrame com o índice
dfs_Regiao.to_csv(os.path.join(download_directory, "Marcas_Regiao.csv"), index=True, index_label="Marca")
print("DataFrame dfs_Regiao salvo como 'Marcas_Regiao.csv'.")

dfs_Tempo.to_csv(os.path.join(download_directory, "Marcas_Tempo.csv"), index=True, index_label="Marca")
print("DataFrame dfs_Tempo salvo como 'Marcas_Tempo.csv'.")

DataFrame dfs_Regiao salvo como 'Marcas_Regiao.csv'.
DataFrame dfs_Tempo salvo como 'Marcas_Tempo.csv'.
